<a href="https://colab.research.google.com/github/springboardmentor441/-IPL_infographics_Infosys_Internship_Oct2024/blob/Sivani/Assignment_1_Sivani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mode

In [ ]:
deliveries=pd.read_csv('deliveries.csv')
matches=pd.read_csv('matches.csv')

In [ ]:
matches.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [ ]:
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


In [ ]:
def normalize(series):
    # Replace infinite values with NaN
    series = series.replace([np.inf, -np.inf], np.nan)

    #Handle the case where the maximum is zero
    series_max = series.max()
    if series_max == 0:
        return series

    return (series / series_max) * 100

In [ ]:
# Calculate Batting Statistics
batting = deliveries.groupby('batter').agg({
    'batsman_runs': 'sum',  # Total runs
    'ball': 'count',        # Total balls faced
    'match_id': 'nunique',  # Matches played
    # Combine centuries and fifties into the same 'batsman_runs' column using functions
    'batsman_runs': [lambda x: x.count(),  # Innings (total times batted)
                    lambda x: (x >= 100).sum(),  # Centuries
                    lambda x: (x >= 50).sum()]  # Fifties
}).reset_index()

In [ ]:
batting.columns

MultiIndex([(      'batter',           ''),
            ('batsman_runs', '<lambda_0>'),
            ('batsman_runs', '<lambda_1>'),
            ('batsman_runs', '<lambda_2>'),
            (        'ball',      'count'),
            (    'match_id',    'nunique')],
           )

In [ ]:
batting = deliveries.groupby('batter').agg(
    total_runs=('batsman_runs', 'sum'),
    balls_faced=('ball', 'count'),
    matches_played=('match_id', 'nunique'),
    innings=('batsman_runs', 'count'),
    centuries=('batsman_runs', lambda x: (x >= 100).sum()),
    fifties=('batsman_runs', lambda x: (x >= 50).sum())
).reset_index()

           batter  total_runs  balls_faced  matches_played  innings  \
0  A Ashish Reddy         280          196              23      196   
1        A Badoni         634          505              35      505   
2      A Chandila           4            7               2        7   
3        A Chopra          53           75               6       75   
4     A Choudhary          25           20               3       20   

   centuries  fifties  
0          0        0  
1          0        0  
2          0        0  
3          0        0  
4          0        0  


In [ ]:
batting.columns

Index(['batter', 'total_runs', 'balls_faced', 'matches_played', 'innings',
       'centuries', 'fifties'],
      dtype='object')

In [ ]:
# Add derived metrics
batting['average'] = batting['total_runs'] / batting['innings']
batting['strike_rate'] = batting['total_runs'] / batting['balls_faced'] * 100

In [ ]:
# Normalize and Calculate Overall Score
batting['normalized_average'] = normalize(batting['average'])
batting['normalized_strike_rate'] = normalize(batting['strike_rate'])
batting['normalized_centuries'] = normalize(batting['centuries'])
batting['normalized_fifties'] = normalize(batting['fifties'])

In [ ]:
weights = {'average': 0.3, 'strike_rate': 0.4, 'centuries': 0.15, 'fifties': 0.05}
batting['overall_score'] = (
    batting['normalized_average'] * weights['average'] +
    batting['normalized_strike_rate'] * weights['strike_rate'] +
    batting['normalized_centuries'] * weights['centuries'] +
    batting['normalized_fifties'] * weights['fifties']
)

In [ ]:
batting.head()

,batter,total_runs,balls_faced,matches_played,innings,centuries,fifties,average,strike_rate,normalized_average,normalized_strike_rate,normalized_centuries,normalized_fifties,overall_score
0,A Ashish Reddy,280,196,23,196,0,0,1.428571,142.857143,47.619048,47.619048,0,0,33.333333
1,A Badoni,634,505,35,505,0,0,1.255446,125.544554,41.848185,41.848185,0,0,29.293729
2,A Chandila,4,7,2,7,0,0,0.571429,57.142857,19.047619,19.047619,0,0,13.333333
3,A Chopra,53,75,6,75,0,0,0.706667,70.666667,23.555556,23.555556,0,0,16.488889
4,A Choudhary,25,20,3,20,0,0,1.250000,125.000000,41.666667,41.666667,0,0,29.166667


In [ ]:
# Sort players by overall score
top_batsmen = batting.sort_values('overall_score', ascending=False)

In [ ]:
top_batsmen.columns

Index(['batter', 'total_runs', 'balls_faced', 'matches_played', 'innings',
       'centuries', 'fifties', 'average', 'strike_rate', 'normalized_average',
       'normalized_strike_rate', 'normalized_centuries', 'normalized_fifties',
       'overall_score'],
      dtype='object')

In [ ]:
# Safe mode function for non-numeric data using pandas
def safe_mode(series):
    try:
        return series.mode()[0]  # Use pandas mode, which works with strings
    except IndexError:  # If no mode exists
        return None

In [ ]:
# Calculate statistics for runs scored
runs_stats = deliveries.groupby('batter')['batsman_runs'].agg(
    mean='mean',
    median='median',
    mode=safe_mode  # Use the safe_mode function
)

In [ ]:
# Filter deliveries where a dismissal occurred
wickets_data = deliveries[~deliveries['player_dismissed'].isna()]

# Calculate total wickets taken by each bowler and most common dismissed player
wickets_stats = wickets_data.groupby('bowler').agg(
    total_wickets=('player_dismissed', 'count'),  # Total dismissals
    most_common_dismissed_player=('player_dismissed', safe_mode)  # Most common dismissed player
)

In [ ]:
# Print the outputs
print("Top Batsmen by Overall Score:")
print(top_batsmen[['batter', 'overall_score']].head(10))

print("\nRun Statistics (Mean, Median, Mode):")
print(runs_stats)

print("\nWicket Statistics (Mean, Median, Mode):")
print(wickets_stats)

Top Batsmen by Overall Score:
              batter  overall_score
312           L Wood      70.000000
97        B Stanlake      58.333333
234  J Fraser-McGurk      51.333333
461    R Sai Kishore      50.555556
629         Umar Gul      47.894737
497         RS Sodhi      46.666667
583    Shahid Afridi      41.086957
222       I Malhotra      40.833333
652         WG Jacks      40.350877
433          PD Salt      39.575758

Run Statistics (Mean, Median, Mode):
                    mean  median  mode
batter                                
A Ashish Reddy  1.428571     1.0     1
A Badoni        1.255446     1.0     1
A Chandila      0.571429     1.0     1
A Chopra        0.706667     0.0     0
A Choudhary     1.250000     1.0     1
...                  ...     ...   ...
Yashpal Singh   0.701493     0.0     0
Younis Khan     0.428571     0.0     0
Yudhvir Singh   1.375000     0.5     0
Yuvraj Singh    1.247848     1.0     0
Z Khan          0.829787     0.0     0

[673 rows x 3 columns]

Wick